In [ ]:
import select
import psycopg2
import psycopg2.extensions

conn = psycopg2.connect(database="twitter", user="postgres")

conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

curs = conn.cursor()
curs.execute("LISTEN new_tweet;")

while 1:
    conn.poll()
    if conn.notifies:
        notify = conn.notifies.pop(0)
        print "Got it"

In [ ]:
from flask import Flask
from flask_sockets import Sockets
from json import dumps
import time

app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    while not ws.closed:
        ws.send('tweet : '+str(j))
        j+=1
        time.sleep(5)
        print ws.closed


if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8091), app, handler_class=WebSocketHandler)
    server.serve_forever()